<center>
<h4>Diplomatura AACSyA 2018 - FaMAF - UNC</h4>
<h1>Aprendizaje No Supervisado</h1>
<h2>Práctico 1 - Clustering</h2>
<hr>
Por David Gonzalez <leonardo.david.gonzalez@gmail.com> y Facundo Díaz Cobos <facundo.diaz.cobos@gmail.com>
</center>
<br/>

## Objetivos
En este práctico se explorarán diferentes soluciones de clustering, para desarrollar las capacidades de análisis de
soluciones de clustering. Es preferible que los conjuntos de datos con los que trabajar sean propios, ya que de esta
forma podrán aplicar su conocimiento del dominio en la interpretación de las diferentes soluciones. Alternativa-
mente, pueden usar conjuntos de datos de los ejemplos de la materia.
En los mismos, hacer una breve discusión del problema y explicar cómo puede ser útil usar técnicas de clustering.

# Consignas
Para cumplir los objetivos, realizar las siguientes actividades:

1 - Explorar soluciones con diferentes parámetros y compararlas. Por ejemplo, variar el número de clusters, las
métricas de distancia, el número de iteraciones o el número de veces que se inicializan las semillas. Describir
brevemente: número de clusters, población de cada cluster, algunas caracterı́sticas distintivas de cada cluster,
algunos elementos que se puedan encontrar en cada cluster.

2 - Incorporar un embedding como preproceso a los datos, aplicar los algoritmos de clustering después de ese
preproceso y describir la solución o soluciones resultantes, discutiendo las ventajas que resultan. Se pueden
usar:

◦ Principal
Component Analysis http://scikit-learn.org/stable/modules/generated/sklearn.
decomposition.PCA.html

◦ para texto, embeddings neuronales Gensim https://pypi.org/project/gensim/

◦ para texto, embeddings neuronales Fastext https://pypi.org/project/fasttext/

3 - Proponer (y en lo posible, implementar) métricas de evaluación de soluciones de clustering basadas en testigos.
Los testigos son pares de objetos que un experto de dominio etiqueta como “deberı́an estar en el mismo cluster”
o “deberı́an estar en distintos clusters”.

4 - El método k-means de scikit-learn no provee una forma sencilla de obtneer los objetos más cercanos al centroide
de un cluster. Proponga alguna forma de obtener una muestra de los elementos de un cluster que sean cercanos
al centroide, por ejemplo, usando clasificadores, usando distancia coseno, etc. En lo posible, implementarlos y
mostrar esos elementos, discutir la representatividad de los elementos encontrados.

# <u>RESOLUCIÓN</u>

# Importando los datos:
Vamos a trabajar un set de datos correspondiente a compras reales de clientes realizadas en el año 2017. Los clientes fueron anonimizados previamente para poder ser utilizados en el ejercicio.

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from yellowbrick.cluster.elbow import KElbowVisualizer
from sklearn.cluster import KMeans
from datetime import datetime

import numpy as np
np.random.seed(0)

# Configuramos el tamaño de los gráficos, en pulgadas
from pylab import rcParams
rcParams['figure.figsize'] = 18, 10

# 2 - Incorporar un embedding como preproceso a los datos:

Intentaremos hacer una reducción de dimensionalidad mediante PCA:


In [ ]:
import pandas as pd
ppcp_log_norm = pd.read_csv('clustering-1-datos-normalizados.csv')

In [ ]:
from IPython.display import display, HTML
# Primero obtenemos la matriz de correlacion 
df_corr = ppcp_log_norm.corr()

display(df_corr.head())
display(df_corr.tail())

In [ ]:
# Lo vemos en el gráfico:
plt.title('Heatmap Correlation - Original',fontsize=15)
plt.imshow(df_corr, cmap=plt.cm.get_cmap("YlGnBu"), interpolation="nearest")
plt.colorbar()
plt.show()

## Revisión 1: Obtener la cantidad de features indicando el porcentaje de varianza

In [ ]:
# Aqui podemos ver que algunas features tiene mucha correlacion entre si
# Por lo que vamos a aplicar una técnica de embedding, PCA (Principal component analisis)
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

pca = PCA(.99)
pca.fit(ppcp_log_norm)
df_reduced_99_pca = pca.transform(ppcp_log_norm)

In [ ]:
# Vamos a volver a generar otro mapa de calor, par ver si la correlacion bajo y el PCA fue efectivo.
df_red_99 =  pd.DataFrame(data=df_reduced_99_pca, columns=range(df_reduced_99_pca.shape[1]))
#Generamos un nuevo mapa de calor para ver corroborar que la correlacion entre columnas es baja!
df_red_99_corr = df_red_99.corr()
# Lo vemos en el gráfico:
plt.title('Heatmap Correlation - PCA',fontsize=15)
plt.imshow(df_red_99_corr, cmap=plt.cm.get_cmap("YlGnBu"), interpolation="nearest")
plt.colorbar()
plt.show()

In [ ]:
# Obtenemos la cantidad de features que mantienen el 99% de la información
HTML(str(df_red_99.shape[1]) + ' features')

## Revisión 2: Obtener la cantidad de features analizando la relación de varianza

In [ ]:
# Detección de cantida de features minima que nos dé la mayor varianza
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Entrenamos con todos los features
n_features = ppcp_log_norm.shape[0]

covar_matrix = PCA(n_components = n_features) 
covar_matrix.fit(ppcp_log_norm)
variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios

var=np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3)*100)
var # cumulative sum of variance explained with [n] features


In [ ]:
# Visualizamos en un grafico:
plt.ylabel('% Variance Explained')
plt.xlabel('# of Features')
plt.title('PCA Analysis')
plt.plot(var)
plt.show()

In [ ]:
# El valor de corte está entre los 370 y 380 feature. Analizamos mas de cerca para ver el punto de corte.
plt.ylabel('% Variance Explained')
plt.xlabel('# of Features')
plt.title('PCA Analysis')

plt.plot(var[370:380])
plt.show()

La gráfica nos muestra que luego de 375 features no mejora la variance. Tomamos esa cantidad para entrenar PCA.

In [ ]:
# Aqui podemos ver que algunas features tiene mucha correlacion entre si
# Por lo que vamos a aplicar una técnica de embedding, PCA (Principal component analisis)
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

pca = PCA(n_components=375)
pca.fit(ppcp_log_norm)
df_reduced_pca = pca.transform(ppcp_log_norm)
# Vamos a volver a generar otro mapa de calor, par ver si la correlacion bajo y el PCA fue efectivo. 1677679
df_red =  pd.DataFrame(data=df_reduced_pca, columns=range(df_reduced_pca.shape[1]))

In [ ]:
display(df_red.head())
display(df_red.tail())

In [ ]:
#Generamos un nuevo mapa de calor para ver corroborar que la correlacion entre columnas es baja!
# y efectivamente este metodo nos ayudo a reducir dimencionalidad sin tanta perdida de informacion
# ya que ahora estamos en otro espacio, sin tanta perdida de informacion
df_red_corr = df_red.corr()

# Lo vemos en el gráfico:
plt.title('Heatmap Correlation - PCA',fontsize=15)
plt.imshow(df_red_corr, cmap=plt.cm.get_cmap("YlGnBu"), interpolation="nearest")
#plt.imshow(df_corr, cmap=plt.cm.get_cmap("Reds"), interpolation="nearest")
plt.colorbar()
plt.show()

In [ ]:
HTML(str(df_red.shape[1]) + ' features')

## Clusterizamos el resultado:
Trabajamos ahora con las columnas detectadas y reclusterizamos para ver si mejora la distribución:

In [ ]:
# Clusterizamos los clientes segun la similaridad de los productos que compran
plt.clf()
clusterClientes = KElbowVisualizer(KMeans(), k=(2,25), metric='silhouette')
clusterClientes.fit(df_red)
clusterClientes.poof()
plt.show()

Ok, probamos con 22 clusters:

In [ ]:
num_clusters = 22

# Ejecutamos la clusterización por la cantidad de clusters seleccionada:
np.random.seed(0)
ppcp_kmeans = KMeans(n_clusters=num_clusters)

ppcp_clusters = ppcp_kmeans.fit_predict(df_red)

pd.DataFrame(ppcp_clusters).hist()
plt.show()


In [ ]:
unique, counts = np.unique(ppcp_clusters, return_counts=True)

for index in range(0, len(unique)):
    display( '* Cantidad de clientes del CLUSTER '+str(index)+': ' + str( counts[index]))

Se observa que los valores de distribución son parecidos al primer cluster. A continuacion probaremos poner algunas metricas de evaluacion basadas en testigos. 

In [ ]:
ppcp_log_norm.to_csv('clustering-1-datos-clusterizados.csv')